In [1]:
# imports
import os, sys
import io
import time
import numpy as np
import xarray as xr
import dask
import dask.array as da
import rasterio
import arcpy

# import tools
#sys.path.append('../../../shared') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
import arc, tools, satfetcher

# import gdvspectra module
#sys.path.append('../../../modules') temp for demo
sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
import cog

# globals 
AWS_KEY = ''
AWS_SECRET = ''
STAC_ENDPOINT = 'https://explorer.sandbox.dea.ga.gov.au/stac/search'
RESULT_LIMIT = 250

In [2]:
# get minimum bounding geom from input 
bbox = [119.32132692558059, -22.774853394924776, 119.34873641649256, -22.745010351081362]

# get collections based on platform 
collections = ['s2a_ard_granule', 's2b_ard_granule']

# prepare start and end date times
in_from_date = '2020-01-01'
in_to_date = '2020-12-31'

in_slc_off = False

RESULT_LIMIT = 250

# fetch stac data 
feats = cog.fetch_stac_data(stac_endpoint=STAC_ENDPOINT, 
                            collections=collections, 
                            start_dt=in_from_date, 
                            end_dt=in_to_date, 
                            bbox=bbox,
                            slc_off=in_slc_off,
                            limit=RESULT_LIMIT)

Beginning STAC search for items. This can take awhile.
Searching collection: s2a_ard_granule
Searching collection: s2b_ard_granule
Sorting result by time (old to new).
Found a total of 148 scenes.


In [3]:
in_bands = 'Blue;Green;Red;NIR1;SWIR2;SWIR3;OA_Mask'
in_platform = 'Sentinel'

# prepare band (i.e. stac assets) names
assets = arc.prepare_band_names(in_bands=in_bands, 
                                in_platform=in_platform)

In [4]:
in_epsg = 3577
in_res = 10
in_snap = True

# convert raw stac into dict with coord reproject, etc.
arcpy.SetProgressorLabel('Converting STAC data into useable format...')
meta, asset_table = cog.prepare_data(feats, 
                                     assets=assets,
                                     bounds_latlon=bbox, 
                                     bounds=None, 
                                     epsg=in_epsg, 
                                     resolution=in_res, 
                                     snap_bounds=in_snap,
                                     force_dea_http=True)

Converting raw STAC data into numpy format.
Converted raw STAC data successfully.


In [5]:
in_resampling = 'Nearest'
in_fill_value = '-999'
in_chunk_size = -1
in_dtype = 'int16'
in_rescale = True

# prepare resample and fill value types
resampling = in_resampling.lower()
fill_value = arc.prepare_fill_value_type(in_fill_value)

# convert assets to dask array
darray = cog.convert_to_dask(meta=meta, 
                             asset_table=asset_table, 
                             chunksize=in_chunk_size,
                             resampling=resampling, 
                             dtype=in_dtype, 
                             fill_value=fill_value, 
                             rescale=in_rescale)

Converting data into dask array.
Converted data successfully.


In [6]:
in_cell_align = 'Top-left'

# prepare alignment type
cell_align = arc.prepare_cell_align_type(in_cell_align)

# generate coordinates and dimensions from metadata
coords, dims = cog.build_coords(feats=feats,
                                assets=assets, 
                                meta=meta,
                                pix_loc=cell_align)

Creating dataset coordinates and dimensions.
Created coordinates and dimensions successfully.


In [7]:
# build final xarray data array
ds_name = 'stac-' + dask.base.tokenize(darray)
ds = xr.DataArray(darray,
                  coords=coords,
                  dims=dims,
                  name=ds_name
                  )

In [8]:
# comvert to cleaner xarray dataset
ds = ds.to_dataset(dim='band')

# append attributes onto dataset
ds = cog.build_attributes(ds=ds,
                          meta=meta, 
                          fill_value=fill_value, 
                          collections=collections, 
                          slc_off=in_slc_off, 
                          bbox=bbox,
                          resampling=in_resampling)

Preparing and appending attributes to dataset.
Attributes appended to dataset successfully.


In [9]:
# get list of dataset vars and iterate compute on each
for counter, data_var in enumerate(list(ds.data_vars), start=1):

    # start clock
    start = time.time()

    # update progress bar
    print('Downloading band: {}...'.format(data_var))

    # compute!
    ds[data_var] = ds[data_var].compute()

    # notify time 
    duration = round(time.time() - start, 2)
    print('Band: {} took: {}s to download.'.format(data_var, duration)) 

Band: nbart_blue took: 87.18s to download.
Band: nbart_green took: 86.12s to download.
Band: nbart_red took: 90.6s to download.
Band: nbart_nir_1 took: 94.79s to download.
Band: nbart_swir_2 took: 45.14s to download.
Band: nbart_swir_3 took: 42.04s to download.
Band: fmask took: 25.12s to download.


In [10]:
ds

<xarray.Dataset>
Dimensions:       (time: 148, x: 311, y: 361)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-02T02:23:55 ... 2020-12-29T02...
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) int16 1277 1332 1475 1537 ... 508 520 508 512
    nbart_green   (time, y, x) int16 1407 1576 1629 1723 ... 724 723 730 728
    nbart_red     (time, y, x) int16 2031 2134 2207 2229 ... 1211 1244 1256 1273
    nbart_nir_1   (time, y, x) int16 2123 2254 2441 2550 ... 1709 1726 1767 1779
    nbart_swir_2  (time, y, x) int16 2978 3179 3179 3307 ... 2637 2637 2661 2661
    nbart_swir_3  (time, y, x) int16 2753 2954 2954 3130 ... 2145 2145 2145 2145
    fmask         (time, y, x) int16 2 2 2 2 2 2 2 2 2 2 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         (10.0, 0.0, -1288220.0, 0.0, -10.0, -2513880.0, 0.0, 0...
    res:               10
    nodatavals:        -999
    orig_collections:  ('s2a_ard_granule', 's2b_ard_granule')
    orig_bbox:         (119.32132692558059, -22.774853394924776, 119.34873641...
    orig_slc_off:      False
    orig_resample:     Nearest

In [13]:
#mask_band = 'oa_fmask'
#cog.remove_fmask_dates(ds, valid_class=[1, 4, 5], max_invalid=5, mask_band='fmask', nodata_value=np.nan, drop_fmask=True)

Removing dates where too many invalid pixels.
Filling invalid pixels with requested nodata value.
Dropping mask band.
Removed invalid images successfully.


<xarray.Dataset>
Dimensions:       (time: 120, x: 311, y: 361)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-12T02:23:55 ... 2020-12-29T02...
  * y             (y) float64 -2.514e+06 -2.514e+06 ... -2.517e+06 -2.517e+06
  * x             (x) float64 -1.288e+06 -1.288e+06 ... -1.285e+06 -1.285e+06
    spatial_ref   int32 3577
Data variables:
    nbart_blue    (time, y, x) float32 518.0 510.0 523.0 ... 520.0 508.0 512.0
    nbart_green   (time, y, x) float32 710.0 705.0 734.0 ... 723.0 730.0 728.0
    nbart_red     (time, y, x) float32 1.299e+03 1.311e+03 ... 1.273e+03
    nbart_nir_1   (time, y, x) float32 1.609e+03 1.641e+03 ... 1.779e+03
    nbart_swir_2  (time, y, x) float32 2.438e+03 2.553e+03 ... 2.661e+03
    nbart_swir_3  (time, y, x) float32 2.128e+03 2.207e+03 ... 2.145e+03
Attributes:
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         (10.0, 0.0, -1288220.0, 0.0, -10.0, -2513880.0, 0.0, 0...
    res:               10
    nodatavals:        -999
    orig_collections:  ('s2a_ard_granule', 's2b_ard_granule')
    orig_bbox:         (119.32132692558059, -22.774853394924776, 119.34873641...
    orig_slc_off:      False
    orig_resample:     Nearest

In [29]:
# get the collection of dataset - will either be
# start with ga_ls or s2 
collections = ds.attrs.get('orig_collections')

# check if tuple, unpack
if isinstance(collections, tuple) and len(collections) > 0:
    in_platform = collections[0]
elif isinstance(collections, str):
    in_platform = collections
else:
    raise ValueError('Input NetCDF missing DEA AWS metadata. Fetch a new cube.')

# parse dea aws platform code from collections attirbute
if in_platform[:5] == 'ga_ls':
    in_platform = 'landsat'
elif in_platform[:2] == 's2':
    in_platform = 'sentinel'
else:
    raise ValueError('Platform in NetCDF is not supported.')

In [45]:
in_nc = r'D:\output\mdr.crf'

In [60]:


# datetime object containing current date and time


print("date and time =", dt)

date and time = 20102021125403


In [67]:
ds.close()

In [65]:


# create unique putput file


In [66]:
out_crf

'D:\\output\\mdr_20102021125723.crf'